<a href="https://colab.research.google.com/github/Subhajit53/Email-Campaign-Effectiveness-Prediction/blob/main/Email_Campaign_Effectiveness_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Email Campaign Effectiveness Prediction : Characterize the mail and track the mail</u></b>

## <b> Problem Description </b>

### Most of the small to medium business owners are making effective use of Gmail-based Email marketing Strategies for offline targeting of converting their prospective customers into leads so that they stay with them in Business. The main objective is to create a machine learning model to characterize the mail and track the mail that is ignored; read; acknowledged by the reader.

## <b> Data Description </b>

### Number of observations : 68353

### <b> Columns : </b>

1.   Email_ID : Unique identifier of emails sent
2.   Email_Type : Type of email encoded as 1 and 2   
3.   Subject_Hotness_Score : A score between 0 to 5 for hotness of the email topic
4.   Email_Source_Type : Source of email encoded as 1 and 2
5.   Customer_Location : Location of customer encoded as A,B,C,D,E,F,G
6.   Email_Campaign_Type : Type of campaign encoded as 1, 2 and 3
7.   Total_Past_Communications : Number of past communications
8.   Time_Email_sent_Category	: Time at which email email was sent encoded as 1, 2 and 3
9.   Word_Count : Words in the email
10.   Total_Links : Number of links in the email
11.   Total_Images : Number of images in the email
12.   Email_Status : Email status encoded as 0 : ignored, 1 : read, 2 : acknowledged









In [1]:
import pandas as pd

In [ ]:
# Mounting drive
from google.colab import drive
drive.mount('/content/drive')

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Classification Project/Copy of data_email_campaign.csv', index_col='Email_ID')

In [5]:
df.head()

,Email_Type,Subject_Hotness_Score,Email_Source_Type,Customer_Location,Email_Campaign_Type,Total_Past_Communications,Time_Email_sent_Category,Word_Count,Total_Links,Total_Images,Email_Status
Email_ID,,,,,,,,,,,
EMA00081000034500,1,2.2,2,E,2,33.0,1,440,8.0,0.0,0
EMA00081000045360,2,2.1,1,NaN,2,15.0,2,504,5.0,0.0,0
EMA00081000066290,2,0.1,1,B,3,36.0,2,962,5.0,0.0,1
EMA00081000076560,1,3.0,2,E,2,25.0,2,610,16.0,0.0,0
EMA00081000109720,1,0.0,2,C,3,18.0,2,947,4.0,0.0,0


In [8]:
df.describe(include='all')

,Email_Type,Subject_Hotness_Score,Email_Source_Type,Customer_Location,Email_Campaign_Type,Total_Past_Communications,Time_Email_sent_Category,Word_Count,Total_Links,Total_Images,Email_Status
count,68353.000000,68353.000000,68353.000000,56758,68353.000000,61528.000000,68353.000000,68353.000000,66152.000000,66676.000000,68353.000000
unique,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,23173,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.285094,1.095481,1.456513,NaN,2.272234,28.933250,1.999298,699.931751,10.429526,3.550678,0.230934
std,0.451462,0.997578,0.498109,NaN,0.468680,12.536518,0.631103,271.719440,6.383270,5.596983,0.497032
min,1.000000,0.000000,1.000000,NaN,1.000000,0.000000,1.000000,40.000000,1.000000,0.000000,0.000000
25%,1.000000,0.200000,1.000000,NaN,2.000000,20.000000,2.000000,521.000000,6.000000,0.000000,0.000000
50%,1.000000,0.800000,1.000000,NaN,2.000000,28.000000,2.000000,694.000000,9.000000,0.000000,0.000000
75%,2.000000,1.800000,2.000000,NaN,3.000000,38.000000,2.000000,880.000000,14.000000,5.000000,0.000000
